## Введение в обработку естественного языка.

### Урок 4. Тематическое моделирование. EM-алгоритм.

Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [1]:
!pip install apache_beam

!pip install annoy

!pip install pymorphy2

!pip install stop_words

!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.9 MB 25.5 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 47 kB 4.6 MB/s 
     |████████████████████████████████| 508 kB 58.2 MB/s 
     |████████████████████████████████| 270 kB 55.2 MB/s 
     |████████████████████████████████| 151 kB 66.1 MB/s 
     |████████████████████████████████| 2.4 MB 42.5 MB/s 
     |████████████████████████████████| 1.0 MB 61.1 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=cdb37dcfb600ebb4506d066b06d5fb3dcf4449c2af09b39747de075ae02a1c00
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=6887611eb1737a95c9db638a58b302a390e389a3a30627ae5cb51a2240fae5db
  Stored in directory: /root/.c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 20.9 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=395176 sha256=4f0957b363c4b8cab2019c70122a252b99a761dda7224e5460674a0a64c72d6d
  Stored in directory: /root/.cache/pip/wheels/81/94/bf/92cb0e4fef8770fe9c6df0ba588fca30ab7c306b6048ae8a54
Successfully built annoy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 8.2 MB 46.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=859f2d9ac13402e671d0b3f1c6460c964da93739ccd604412d201265a8cc053e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce1505

In [2]:
import numpy as np

import pandas as pd

import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from pymystem3 import Mystem

import pymorphy2

from gensim.models import *
from gensim import corpora

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


**Произведем загрузку данных.**

In [3]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-09-03 16:48:19--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-09-03 16:48:19--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc1394389a02dd699e52b575ede.dl.dropboxusercontent.com/cd/0/inline/BsS1cD-gf-VLuOJNgpxgU57m0uBYnH_TcR3xtQuIyYVJPQkUgQlRDVo-liq7wIQ59iZ47qBT5G8wDoCNUTdTwm2n41f-fKkcY5AqTRLvUI6_Y-eVdo0HaEgcSZMNJZIUvpWgru8iJE6mz87tBIPn6LiEHAXKMz9dgkM0Tzam0jKWCw/file# [following]
--2022-09-03 16:48:19--  https://ucc1394389a02dd699e52b575ede.dl.dropboxusercontent.com/cd/0/inline/BsS1cD-gf-VLuOJNgpxgU57m0uBYnH_TcR3xtQuIyYVJPQkUgQlRDVo-liq7wIQ59iZ47qBT5G8wDoCN

In [4]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative], ignore_index=True)

df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


**Проведем предобработку данных.**

In [5]:
words_regex = re.compile('\w+')

In [6]:
def find_words(text, regex = words_regex):
    tokens = regex.findall(text)
    return [w for w in tokens if w.isalpha() and len(w)>= 4] 

In [7]:
morph = pymorphy2.MorphAnalyzer() 
stopwords_list = stopwords.words('russian') 

In [8]:
def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] 
    return [w for w in lemmas if not w in stopwords and w.isalpha()] 

In [9]:
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

In [10]:
def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

In [11]:
df['text'] = df['text'].apply(preprocess)

df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


**Модель Latent Dirichlet Allocation (LDA).**

Создадим словарь, состоящий из уникальных токенов датасета.

In [12]:
dictionary = corpora.Dictionary(df['text']) 

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) 

dictionary.save('tweet.dict')

len(dictionary.keys()) 

8172

Векторизуем твиты.

In [13]:
corpus = [dictionary.doc2bow(text) for text in df['text']]
 
corpora.MmCorpus.serialize('tweet.model', corpus) 

In [14]:
mm = corpora.MmCorpus('tweet.model') 

print(df['text'].values[0])

print(mm[0]) 

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


Произведем обучение модели.

In [15]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

Определим вероятные темы.

In [16]:
lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.024*"человек" + 0.020*"больш" + 0.017*"жизнь" + 0.017*"новый" + 0.013*"друг" + 0.011*"год" + 0.010*"голова" + 0.009*"найти" + 0.009*"остаться" + 0.008*"телефон" + 0.008*"прийтись" + 0.007*"хороший" + 0.007*"бояться" + 0.007*"рука" + 0.006*"спасибо" + 0.006*"мало" + 0.006*"слово" + 0.006*"фотка" + 0.006*"парень" + 0.005*"твит"'),
 (1,
  '0.038*"день" + 0.023*"сегодня" + 0.014*"время" + 0.013*"жаль" + 0.012*"утро" + 0.012*"последний" + 0.010*"неделя" + 0.010*"хороший" + 0.009*"настроение" + 0.009*"каждый" + 0.009*"экзамен" + 0.008*"самый" + 0.008*"каникулы" + 0.007*"закончиться" + 0.007*"вроде" + 0.007*"любимый" + 0.006*"твиттер" + 0.006*"пока" + 0.006*"нужный" + 0.006*"ужасный"'),
 (2,
  '0.029*"завтра" + 0.022*"дом" + 0.019*"школа" + 0.015*"сегодня" + 0.012*"идти" + 0.012*"никто" + 0.011*"мама" + 0.011*"час" + 0.010*"работа" + 0.009*"ночь" + 0.008*"прийти" + 0.008*"урок" + 0.007*"втор" + 0.007*"улица" + 0.007*"фильм" + 0.007*"нравиться" + 0.006*"снег" + 0.006*"правда" + 0.006

Оценим качество модели.

In [17]:
print(lda.log_perplexity(corpus))

-8.005068308072145


In [18]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.00033376671935011863


**Визуализируем предсказания модели LDA.**

In [19]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)

pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


**Произведем интерпретацию тем.**

Судя по полученным результатам, твиты можно распределить на три группы:

1. обучение в школе;

2. работа;

3. положительные эмоции.